In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [4]:
DATASET_LOCATION = '/content/drive/MyDrive/ai-playground/ra-text-classification-spectrogram/images/isot-4000-bert'
data_dir = pathlib.Path(DATASET_LOCATION)

In [3]:
TRAIN_FRACTION = 0.8
batch_size = 32 # default batch size of data
img_height = 224 # default height to which image is resized
img_width = 224 # default width to which image is resized
seed = 2022 # random seed for shuffling and transformations

In [7]:
ds_train = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split = 1 - TRAIN_FRACTION,
  subset = "training",
  seed = seed,
  image_size = (img_height, img_width),
  batch_size = batch_size)

Found 2332 files belonging to 2 classes.
Using 1866 files for training.


In [8]:
ds_val = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split = 1 - TRAIN_FRACTION,
  subset = "validation",
  seed = seed,
  image_size = (img_height, img_width),
  batch_size = batch_size)

Found 2332 files belonging to 2 classes.
Using 466 files for validation.


In [10]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = ds_train.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = ds_val.cache().prefetch(buffer_size=AUTOTUNE)

In [11]:
normalization_layer = layers.Rescaling(1./255)


In [12]:
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

0.0044817757 1.0


In [15]:
class_names = ds_train.class_names
num_classes = len(class_names)
print(class_names)

['fake', 'true']


In [19]:
model = Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(1, activation='sigmoid')
])

In [20]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.binary_crossentropy,
              metrics=['accuracy'],)

In [21]:
epochs=100
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

Epoch 1/100
59/59 [==============================] - 60s 1s/step - loss: 0.6394 - accuracy: 0.6629 - val_loss: 0.4075 - val_accuracy: 0.8069
Epoch 2/100
59/59 [==============================] - 58s 991ms/step - loss: 0.3138 - accuracy: 0.8767 - val_loss: 0.2364 - val_accuracy: 0.9034
Epoch 3/100
59/59 [==============================] - 65s 1s/step - loss: 0.1918 - accuracy: 0.9228 - val_loss: 0.1837 - val_accuracy: 0.9356
Epoch 4/100
59/59 [==============================] - 59s 994ms/step - loss: 0.1677 - accuracy: 0.9298 - val_loss: 0.1899 - val_accuracy: 0.9227
Epoch 5/100
59/59 [==============================] - 60s 1s/step - loss: 0.1335 - accuracy: 0.9469 - val_loss: 0.1417 - val_accuracy: 0.9549
Epoch 6/100
59/59 [==============================] - 59s 1s/step - loss: 0.1180 - accuracy: 0.9544 - val_loss: 0.1486 - val_accuracy: 0.9464
Epoch 7/100
59/59 [==============================] - 59s 1s/step - loss: 0.1014 - accuracy: 0.9689 - val_loss: 0.1178 - val_accuracy: 0.9549
Epoch 8

KeyboardInterrupt: ignored

In [22]:
from matplotlib import pyplot as plt

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('vgg 16 model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

NameError: ignored